In [3]:
%matplotlib inline
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../../../../')
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from PyFin.api import *
from alphamind.api import *
from conf.models import *
from conf.config import*
from data.engines.model import Record
from alphamind.execution.naiveexecutor import NaiveExecutor
from stacking import factor_store, feature_list
from optimization.bayes_optimization_xgb import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

from data.engines.sqlengine import SQLEngine

weekly = 'st'  # s1, s2,  s3, s4, s5, s_1
record_engine = SQLEngine('sqlite:///./{}/real_tune_record_without_alpha.db'.format(weekly))


In [4]:
import datetime
# ref_date_pre = '2019-12-27'

previous_pos = record_engine.fetch_record_meta(Record, trade_date=None)
previous_pos

,trade_date,adjust_date,weight,industry,er,code
0,2019-12-11 00:00:00.000000,2019-12-11,0.027472,食品饮料,0.005474,860
1,2019-12-11 00:00:00.000000,2019-12-11,0.027298,计算机,0.018910,2405
2,2019-12-11 00:00:00.000000,2019-12-11,0.026296,电子,0.054431,603501
3,2019-12-11 00:00:00.000000,2019-12-11,0.025856,机械设备,0.012662,300450
4,2019-12-11 00:00:00.000000,2019-12-11,0.025835,电子,0.029716,600584
5,2019-12-11 00:00:00.000000,2019-12-11,0.025242,通信,0.015538,300383
6,2019-12-11 00:00:00.000000,2019-12-11,0.025068,医药生物,0.008146,2821
7,2019-12-11 00:00:00.000000,2019-12-11,0.024864,商业贸易,0.019404,2127
8,2019-12-11 00:00:00.000000,2019-12-11,0.024496,传媒,0.027699,300315
9,2019-12-11 00:00:00.000000,2019-12-11,0.024475,电子,0.032149,2916


In [ ]:
pos_record = record_engine.fetch_record('pos_record')
# previous_pos = pos_record[pos_record['trade_date'] == ref_date_pre]
# previous_pos
pos_record